In [11]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, random_split

In [2]:
training_file = 'TrainingData.pkl'
testing_file= 'TestData.pkl'
# testing_file = ?

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
# with open(validation_file, mode='rb') as f:
#     valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = torch.tensor(train['features']), torch.tensor(train['labels'])
# X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = torch.tensor(test['features']), torch.tensor(test['labels'])

In [12]:
transform = transforms.Compose([transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  
])

dataset = datasets.ImageFolder(root='Training', transform=transform)
total = len(dataset)
train_c = 0.85 * len(dataset)
valid_c = 0.15 * len(dataset)

train_data, valid_data = random_split(dataset,[int(train_c),int(valid_c)])
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64,shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=True)

## Build Model

In [7]:
def convertY(y,classes):
    Y = np.zeros((classes,(y.shape[0])))
    for i in range(y.shape[0]):
        Y[y[i]][i] = 1
    return Y 

import random
index = [i for i in range(y_train.shape[0])] 
random.shuffle(index)
X_train = X_train[index]
y_train = y_train[index]

# index = [i for i in range(y_valid.shape[0])] 
# random.shuffle(index)
# X_valid = X_valid[index]
# y_valid = y_valid[index]
X_valid = X_train[0:2500]
y_valid = y_train[0:2500]
X_train = X_train[2500:y_train.shape[0]]
y_train= y_train[2500:y_train.shape[0]]
y_train_hot = convertY(y_train,43)
y_valid_hot = convertY(y_valid,43)

X_train_normalized = (X_train - 128)/128 
X_valid_normalized = (X_valid - 128)/128

In [8]:
class CNN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.cnn1 = nn.Conv2d(in_channels= 3, out_channels= 8, kernel_size= 2, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride = 1,padding=0)

        self.cnn2 = nn.Conv2d(in_channels= 8, out_channels= 16, kernel_size= 2, padding=2, stride=1 )
        self.maxpool2 = nn.MaxPool2d(kernel_size=4, stride=1)

        self.fc1 = nn.Linear(32*32,43)
    

    def forward(self,x):
        x = self.cnn1(x)
        x = torch.relu(x)
        x = self.maxpool1(x)

        x = self.cnn2(x)
        x = torch.relu(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x
    
    def activations(self,x):
        z1 = self.cnn1(x)
        a1 = torch.relu(x)
        out = self.maxpoo1(a1)

        z2 = self.cnn2(out)
        a2 = torch.relu(z2)
        out = out.view(out.size(0),-1)
        return z1,a1, z2, a2, out


In [21]:
model = CNN()
model

CNN(
  (cnn1): Conv2d(3, 8, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1), padding=(2, 2))
  (maxpool2): MaxPool2d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=43, bias=True)
)

In [12]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

RuntimeError: a Tensor with 74158080 elements cannot be converted to Scalar

In [14]:
n_epochs=10
cost_list=[]
accuracy_list=[]
N_test=len(training_file)
cost=0
#n_epochs
for epoch in range(n_epochs):
    cost=0    
    for x, y in (X_train_normalized, y_train):
      

        #clear gradient 
        optimizer.zero_grad()
        #make a prediction 
        z=model(x)
        # calculate loss 
        loss=criterion(z,y)
        # calculate gradients of parameters 
        loss.backward()
        # update parameters 
        optimizer.step()
        cost+=loss.item()
    cost_list.append(cost)
        
        
    correct=0
    #perform a prediction on the validation  data  
    for x_test, y_test in X_valid_normalized, y_valid:

        z=model(x_test)
        _,yhat=torch.max(z.data,1)

        correct+=(yhat==y_test).sum().item()
        

    accuracy=correct/N_test

    accuracy_list.append(accuracy)

ValueError: too many values to unpack (expected 2)